**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [0]:
!pip install -q scikit-video
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd,Adam
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization,Flatten,Dropout,LeakyReLU

# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [0]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [0]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The agent has to explore different scenarios by visiting many states since if it acts only accordingly to a strategy which maximizes its rewards it cannot discover or learn another possible better path. It is then essential to take random actions to do so (and we know that we choose a random action with a probability $\epsilon$ ). This $\epsilon$ helps to capture the exploration/exploitation dilemma.


***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [0]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [0]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=100 # set small when debugging
epochs_test=10 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

#### Position : 
Enables to know the position of the rat . To get more in the details, position is a matrix with size (grid+4)*(grid4) with values of 0 except where the rat is located. The values are -1 on the limits of the maps (forbidden positions for the rat).
#### Board:
Enables to know the configuration of the boards that is to say where the bonus and malus are.  It represents then the repartition of the cheese and the poison on the map. After cheese or poison is collected, the value on the corresponding cell is set to 0.




## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [0]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(0,self.n_action,1)


In [0]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        
        state = env.reset()
       
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent makes an action
            action = agent.act(state)
            
            # We get a reward and get to the next step if the game is not over
            prev_state = state
            state, reward, game_over = env.act(action)
            
            #Updating rewards
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

                
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))
        
    



In [56]:
# Initialize the game

env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 14.5/15.0. Average score (-0.5)
Win/lose count 12.0/11.0. Average score (0.25)
Win/lose count 11.0/17.0. Average score (-1.8333333333333333)
Win/lose count 12.0/10.0. Average score (-0.875)
Win/lose count 11.5/13.0. Average score (-1.0)
Win/lose count 10.0/19.0. Average score (-2.3333333333333335)
Win/lose count 7.5/11.0. Average score (-2.5)
Win/lose count 14.5/19.0. Average score (-2.75)
Win/lose count 9.0/12.0. Average score (-2.7777777777777777)
Win/lose count 9.0/11.0. Average score (-2.7)
Final score: -2.7


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [0]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m)
        while len(self.memory) > self.max_memory:
            self.memory.pop(0)

    def random_access(self):
        return self.memory[np.random.randint(0, len(self.memory), size=1)[0]]

***
The pipeline we will use for training is given below:

In [0]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score =score + win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [0]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(np.expand_dims(s, axis = 0)))

    def reinforce(self, s_, n_s_, a_, r_, game_over_):

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            [s, ns, a, r, game_over] = self.memory.random_access()
            input_states[i] = s
            target_q[i] = self.model.predict(s[np.newaxis])
            if game_over_:
                target_q[i, a] = r
            else:
                target_q[i, a] = r + self.discount * np.max(self.model.predict(ns[np.newaxis]))
        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, layers = 2, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        
        model = Sequential()
        model.add(Reshape((5*5*self.n_state,), input_shape=(5,5,self.n_state)))#reshape because the input is a tensor
        model.add(Dense(64, activation='relu'))
        model.add(Dense(128, activation='relu'))
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.1), "mse")
        self.model = model

In [61]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

Epoch 000/100 | Loss 0.2604 | Win/lose count 4.0/5.0 (-1.0)
Epoch 001/100 | Loss 1.7690 | Win/lose count 3.0/1.0 (2.0)
Epoch 002/100 | Loss 1.8574 | Win/lose count 1.5/2.0 (-0.5)
Epoch 003/100 | Loss 1.9426 | Win/lose count 3.5/1.0 (2.5)
Epoch 004/100 | Loss 1.8374 | Win/lose count 3.0/1.0 (2.0)
Epoch 005/100 | Loss 1.8432 | Win/lose count 6.0/9.0 (-3.0)
Epoch 006/100 | Loss 1.9967 | Win/lose count 5.0/2.0 (3.0)
Epoch 007/100 | Loss 1.9377 | Win/lose count 2.5/5.0 (-2.5)
Epoch 008/100 | Loss 1.8952 | Win/lose count 2.5/3.0 (-0.5)
Epoch 009/100 | Loss 1.9066 | Win/lose count 7.0/7.0 (0.0)
Epoch 010/100 | Loss 1.9717 | Win/lose count 2.5/3.0 (-0.5)
Epoch 011/100 | Loss 2.0675 | Win/lose count 8.5/1.0 (7.5)
Epoch 012/100 | Loss 1.9546 | Win/lose count 3.0/4.0 (-1.0)
Epoch 013/100 | Loss 1.9150 | Win/lose count 7.5/4.0 (3.5)
Epoch 014/100 | Loss 1.8626 | Win/lose count 2.0/4.0 (-2.0)
Epoch 015/100 | Loss 1.8688 | Win/lose count 4.5/4.0 (0.5)
Epoch 016/100 | Loss 1.8548 | Win/lose count 6.0

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [0]:

class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        model = Sequential()
        model.add(Conv2D(32, (2, 2), input_shape=(5,5,self.n_state,), activation='relu'))
        model.add(Dropout(rate=0.2))
        model.add(Conv2D(64, (2, 2), activation='relu'))
        model.add(Flatten())
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.1), "mse")
        self.model = model

In [63]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/100 | Loss 0.0133 | Win/lose count 3.0/2.0 (1.0)
Epoch 001/100 | Loss 0.0294 | Win/lose count 3.5/5.0 (-1.5)
Epoch 002/100 | Loss 0.0947 | Win/lose count 4.0/2.0 (2.0)
Epoch 003/100 | Loss 0.1488 | Win/lose count 0.5/3.0 (-2.5)
Epoch 004/100 | Loss 0.2370 | Win/lose count 6.5/6.0 (0.5)
Epoch 005/100 | Loss 0.4092 | Win/lose count 5.0/2.0 (3.0)
Epoch 006/100 | Loss 0.9325 | Win/lose count 8.5/5.0 (3.5)
Epoch 007/100 | Loss 1.4327 | Win/lose count 7.0/5.0 (2.0)
Epoch 008/100 | Loss 1.4800 | Win/lose count 8.5/2.0 (6.5)
Epoch 009/100 | Loss 1.1914 | Win/lose count 1.0/3.0 (-2.0)
Epoch 010/100 | Loss 1.6152 | Win/lose count 7.0/7.0 (0.0)
Epoch 011/100 | Loss 1.5591 | Win/lose count 3.5/6.0 (-2.5)
Epoch 012/100 | Loss 1.3822 | Win/lose count 10.0/0 (10.0)
Epoch 013/100 | Loss 1.5172 | Win/lose count 2.0/2.0 (0.0)
Epoch 014/100 | Loss 1.4160 | Win/lose count 5.5/6.0 (-0.5)
Epoch 015/100 | Loss 1.4496 | Win/lose count 12.0/2.0 (10.0)
Epoch 016/100 | Loss 1.5676 | Win/lose count 15.5

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [64]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 16.5/3.0. Average score (13.5)
Win/lose count 14.0/7.0. Average score (10.25)
Win/lose count 13.5/0. Average score (11.333333333333334)
Win/lose count 9.0/2.0. Average score (10.25)
Win/lose count 19.5/5.0. Average score (11.1)
Win/lose count 11.0/4.0. Average score (10.416666666666666)
Win/lose count 13.0/1.0. Average score (10.642857142857142)
Win/lose count 8.5/2.0. Average score (10.125)
Win/lose count 16.0/3.0. Average score (10.444444444444445)
Win/lose count 9.0/1.0. Average score (10.2)
Final score: 10.2
Test of the FC
Win/lose count 3.5/2.0. Average score (1.5)
Win/lose count 1.0/2.0. Average score (0.25)
Win/lose count 4.0/6.0. Average score (-0.5)
Win/lose count 2.5/6.0. Average score (-1.25)
Win/lose count 2.0/3.0. Average score (-1.2)
Win/lose count 1.5/6.0. Average score (-1.75)
Win/lose count 3.5/3.0. Average score (-1.4285714285714286)
Win/lose count 2.5/4.0. Average score (-1.4375)
Win/lose count 3.5/6.0. Average score (-1.555555555555555

In [0]:
HTML(display_videos('cnn_test10.mp4'))

In [0]:
HTML(display_videos('fc_test10.mp4'))

The CNN seems to perform better than the neural network. With both methods , the rat used to oscillate much more around some positions to avoid punitions. They seem not to explore more to check if there is remaining cheese (there is no incentive discover new areas of the map). The agent seems also to learn to stay away from posion and hence cannot get to a cheese cell surrounded by poison cells.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [0]:
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state, train = True)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train = True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

            # Update the epsilon for the epsilong greedy policy
            agent.set_epsilon(np.exp(-e/40))
            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.visited = np.zeros((grid_size, grid_size))
        
        # coordinate of the rat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 1] = 256
        b[self.visited>0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]= 0
        b[:,-2:,:]= 0
        b[:2,:,:]= 0
        b[:,:2,:]= 0

        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train = False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        
        reward = 0
        #During training we penalise going back to a visited state
        if train:
            reward = - self.visited[self.x, self.y] * 0.1
        #We tag the new position as visited
        self.visited[self.x, self.y] = 1
        reward = reward + self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1),
                        self.visited[:,:,np.newaxis]),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        #We tag the initial position as visited
        self.visited = np.zeros((self.grid_size, self.grid_size))
        self.visited[self.x, self.y] = 1
        self.t = 0

        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1),
                        self.visited[:,:,np.newaxis]),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

In [66]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/100 | Loss 0.0113 | Win/lose count 2.5/26.000000000000103 (-23.500000000000103)
Epoch 001/100 | Loss 0.0161 | Win/lose count 3.5/31.000000000000107 (-27.500000000000107)
Epoch 002/100 | Loss 0.0097 | Win/lose count 7.0/24.000000000000043 (-17.000000000000043)
Epoch 003/100 | Loss 0.0129 | Win/lose count 9.0/30.400000000000105 (-21.400000000000105)
Epoch 004/100 | Loss 0.0435 | Win/lose count 17.5/25.300000000000054 (-7.800000000000054)
Epoch 005/100 | Loss 0.4524 | Win/lose count 10.0/30.90000000000014 (-20.90000000000014)
Epoch 006/100 | Loss 1.5767 | Win/lose count 13.0/27.600000000000094 (-14.600000000000094)
Epoch 007/100 | Loss 1.5804 | Win/lose count 11.0/21.800000000000054 (-10.800000000000054)
Epoch 008/100 | Loss 1.4914 | Win/lose count 13.5/28.100000000000023 (-14.600000000000023)
Epoch 009/100 | Loss 1.5490 | Win/lose count 19.0/21.900000000000016 (-2.9000000000000163)
Epoch 010/100 | Loss 1.5634 | Win/lose count 15.5/27.100000000000062 (-11.600000000000062)
Epoch 

In [67]:
env = EnvironmentExploring(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_train_exploremodel.h5',name_model='cnn_train_exploremodel.json')
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore9.mp4'))

Win/lose count 25.5/4.0. Average score (21.5)
Win/lose count 26.0/3.0. Average score (22.25)
Win/lose count 25.5/4.0. Average score (22.0)
Win/lose count 26.0/6.0. Average score (21.5)
Win/lose count 23.0/7.0. Average score (20.4)
Win/lose count 18.0/3.0. Average score (19.5)
Win/lose count 25.5/3.0. Average score (19.928571428571427)
Win/lose count 15.0/2.0. Average score (19.0625)
Win/lose count 22.5/3.0. Average score (19.11111111111111)
Win/lose count 25.0/6.0. Average score (19.1)
Final score: 19.1


***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***